In [6]:
# GeneralCNNs测试
import os
import torch
from models.supervisedModels.CNNs import GeneralCNNs
from utils.pytorch_tools.model_analysis_tools import DualBranchModelTools
network_branch = 1
gait_or_motion = 'motion'
motion_type = 'WAK'
conv_type = 'SKConv' # one of [DNN, Conv, SKConv, ResNetV1, ResNetV2, MobileNetV1, MobileNetV2, MobileNetV3, ResNeSt, ShuffleNetV1, ShuffleNetV2]
model = GeneralCNNs(network_branch, gait_or_motion, motion_type, conv_type=conv_type)
model_name = model.get_model_name()
print(model_name)
input_data_size = (1, 15, 96)
input_feature_size = (1, 15, 6)
input_size = [input_data_size, input_feature_size]
batch_size = 32
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model_tools = DualBranchModelTools(model, input_size, batch_size, device)
# 注意顺序，如果使用plot_model，必须放在calculate_complexity前面
model_tools.test_output_shape()
model_tools.summary_model(method='summary') #'summary' / 'fvcore'
model_tools.plot_model(model_name=model_name, save_format='pdf', show=True, verbose=False)
macs, params = model_tools.calculate_complexity(verbose=True, print_log=True)
# 因为使用了thop,去获取整个model 的mac，所以会导致total_params以及total_ops被注册到网络中，所以会导致miss错误，所以之后要在实例化一次模型。
model = GeneralCNNs(network_branch, gait_or_motion, motion_type, conv_type=conv_type)

OneBranchCNN-SKConv

================================================================================2023-10-08 17:44:17
测试模型输出维度...

output.shape torch.Size([32, 5])

================================================================================2023-10-08 17:44:17
使用torchsummary打印模型每个网络层的形状...

Layer (type:depth-idx)                             Output Shape              Param #
├─Sequential: 1-1                                  [-1, 96, 15, 1]           --
|    └─SKBlock: 2-1                                [-1, 32, 15, 24]          --
|    |    └─SKConvBlock: 3-1                       [-1, 32, 15, 96]          --
|    |    |    └─ModuleList: 4                     []                        --
|    |    |    |    └─Sequential: 5-1              [-1, 32, 15, 96]          --
|    |    |    |    |    └─Conv2d: 6-1             [-1, 32, 15, 96]          128
|    |    |    |    |    └─BatchNorm2d: 6-2        [-1, 32, 15, 96]          64
|    |    |    |    |    └─ReLU: 6-3               [-1,

In [5]:
# GeneralGNNs测试
import os
import torch
from models.supervisedModels.GNNs import GeneralGNNs
from utils.pytorch_tools.model_analysis_tools import DualBranchModelTools
# 模型设置
network_branch = 2  #  1) network_branch = 1，只使用原始数据; 2)  network_branch = 2，同时使用原始数据和时域特征.
gnn_mode = 'GATv2Conv'
# 空间域： 1）GATConv (2017); 2) TransformerConv (2020); 3) GATv2Conv (2021); 4) SuperGATConv (2021)
# 谱域： 5）GCNConv (2016); 6) ChebConv (2016);  7) LEConv (2019); 8)SSGConv (2021).
edge_gen_mode = 'TRG' # 1) 'PSK'; 2) 'PCC'; 3) 'KNN'; 4) 'TRG'; 5) 'AWMF'.
edge_weight_mode = 'default' # 1) 'default'; 2) 'learnable'.
graph_fusion_mode = 'default' # 1) 'default'; 2) 'learnable'.
readout_mode = 'fc' # 1) 'max'; 2) 'mean'; 3) 'fc'.
conv_type = 'MobileNetV3' # one of [Conv, SKConv, ResNetV1, ResNetV2, MobileNetV1, MobileNetV2, MobileNetV3]'
node_amount = 15
gnn_params = {'max_esr_ratio': 0.2, 'trg_edge_connect_mode': 'node_random', 'node_embedding': False, 'kgnn_ratio': 0.5, 'edge_act_thr': 0.5}
#  params: 1) max_esr_ratio: TRG图中保留的最大边比例 max_edge_number = math.floor(max_esr_ratio*node_amount); 
#          2) trg_edge_connect_mode: 'graph_fix' / 'node_fix' 则为max_edge_number; 'node_random'则为random.choice(list(range(1, max_edge_number)))
#          3) node_embedding: 节点嵌入，对'node_fix'和'node_random'，设为False较好；对'graph_fix'，可尝试设为True 
#          4) kgnn_ratio: KNN图中K近邻的比例; 4) edge_act_thr: 用于控制边连接是否激活的阈值，影响PCC，AWMF
# 数据设置， for num_classes计算
gait_or_motion = 'motion' #  1) 'gait'; 2)'motion'
motion_type = 'UPS'
# 1) gait_or_motion = 'motion'时， motion_type可为任何字符串；
# 2) gait_or_motion = 'gait'时， motion_type为：’WAK', 'UPS', 'DNS'
# 数据设置， for psk邻接矩阵的路径
# 获取当前文件所在目录os.getcwd()的上一级目录os.path.dirname(),即’models'，再进入'models\commonBlocks'
psk_path = os.path.join(os.path.dirname(os.getcwd()) , 'commonBlocks')
model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                     conv_type, node_amount, gait_or_motion, motion_type, psk_path, gnn_params)
model_name = model.get_model_name()
print(model_name)
input_data_size = (1, 15, 96)
input_feature_size = (1, 15, 6)
input_size = [input_data_size, input_feature_size]
batch_size = 10
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model_tools = DualBranchModelTools(model, input_size,  batch_size, device)
# 注意顺序，如果使用plot_model，必须放在calculate_complexity前面
model_tools.test_output_shape()
model_tools.summary_model(method='summary') #'summary' / 'fvcore'
# model_tools.plot_model(model_name=model_name, save_format='pdf', show=True, verbose=False)
macs, params = model_tools.calculate_complexity(verbose=True, print_log=True)
# 因为使用了thop,去获取整个model 的mac，所以会导致total_params以及total_ops被注册到网络中，所以会导致miss错误，所以之后要在实例化一次模型。
model = GeneralGNNs(network_branch, gnn_mode, edge_gen_mode, edge_weight_mode, graph_fusion_mode, readout_mode,
                     conv_type, node_amount, gait_or_motion, motion_type, psk_path, gnn_params)

TwoBranchGNN-MobileNetV3-TRG-default-GATv2Conv

================================================================================2023-10-12 17:04:01
测试模型输出维度...

output.shape torch.Size([10, 5])

================================================================================2023-10-12 17:04:01
使用torchsummary打印模型每个网络层的形状...

Layer (type:depth-idx)                                  Output Shape              Param #
├─Sequential: 1-1                                       [-1, 96, 15, 1]           --
|    └─CNNBlock: 2-1                                    [-1, 32, 15, 24]          --
|    |    └─Conv2d: 3-1                                 [-1, 32, 15, 96]          192
|    |    └─BatchNorm2d: 3-2                            [-1, 32, 15, 96]          64
|    |    └─ReLU: 3-3                                   [-1, 32, 15, 96]          --
|    |    └─MaxPool2d: 3-4                              [-1, 32, 15, 24]          --
|    |    └─Dropout: 3-5                                [-1, 32, 15, 24] 